In [23]:
import requests
from bs4 import BeautifulSoup as bs
url = "https://www.ptt.cc/bbs/Gossiping/index.html"


prefix = "https://www.ptt.cc/"
# cookies = {
#     "over18":"1"
# }
# #發送網頁請求
r = requests.get(url)

soup = bs(r.text, 'lxml')
# containers = soup.select(".r-ent").select(".title")
# containers = soup.select(".r-ent .title a")
# print(containers[0]["href"])

title_elements = soup.select(".r-ent .title a")


for ele in title_elements:
    print(f"標題: {ele.get_text()}網址:{prefix+ele["href"]}")

    ptt_response = requests.get(prefix + ele['href'])

    ptt_response

標題: Re: [問卦] 沒想到館長人格有小瑕疵!很扯!!!網址:https://www.ptt.cc//bbs/Gossiping/M.1763186667.A.4D9.html
標題: [新聞] 蘋果推高價「手機背帶」遭網路嘲諷！一網址:https://www.ptt.cc//bbs/Gossiping/M.1763186715.A.159.html
標題: [問卦] 光電板毀台環團沒出來，修法環評卻出來了網址:https://www.ptt.cc//bbs/Gossiping/M.1763186811.A.FD4.html
標題: [問卦] 你們網紅圈真的很亂網址:https://www.ptt.cc//bbs/Gossiping/M.1763186812.A.F7D.html
標題: [新聞] 美韓貿易、安保達共識 准建核潛艦  網址:https://www.ptt.cc//bbs/Gossiping/M.1763186861.A.108.html
標題: [問卦] Toyz的拾玖茶屋會推新品嗎網址:https://www.ptt.cc//bbs/Gossiping/M.1763186878.A.EE8.html
標題: [問卦] Wemb斑馬 因該超越MJ了吧?網址:https://www.ptt.cc//bbs/Gossiping/M.1763186969.A.1EC.html
標題: [新聞] 美軍被國防商當盤子！陸軍部長曝：黑鷹零網址:https://www.ptt.cc//bbs/Gossiping/M.1763187029.A.690.html
標題: [問卦] 漁港代客料理的廚師 是不是都很厲害？網址:https://www.ptt.cc//bbs/Gossiping/M.1763187089.A.263.html
標題: Re: [新聞] 遭中共通緝！沈伯洋沒在怕　現身德國國網址:https://www.ptt.cc//bbs/Gossiping/M.1763187171.A.4CD.html
標題: [問卦] 館長4真男人吧網址:https://www.ptt.cc//bbs/Gossiping/M.1763187191.A.F2C.html
標題: [問卦] 中天蔡董會幫館長才有鬼吧網址:https://www.ptt.cc//bbs/G

In [29]:
import requests
from bs4 import BeautifulSoup as bs
import re

url = "https://tw.stock.yahoo.com/quote/2330.TW"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

r = requests.get(url, headers=headers)
soup = bs(r.text, "lxml")

# ---------- 股名：從 <title> 抓 ----------
page_title = soup.title.get_text(strip=True)   # ex: "台積電(2330.TW) 走勢圖 - Yahoo奇摩股市"
stock_name = page_title.split("(")[0]          # -> "台積電"

# ---------- 股價：從整頁文字裡抓「成交 1,430」 ----------
full_text = soup.get_text(" ", strip=True)
# 這裡會去找類似「成交 1,430」這種型態
m = re.search(r"成交\s+([\d,]+(?:\.\d+)?)", full_text)

stock_price = m.group(1) if m else None

print(f"股名：{stock_name}")
print(f"股價：{stock_price}")


股名：台積電
股價：1,430


In [74]:

def get_stock_info(code):
    url = f"https://tw.stock.yahoo.com/quote/{code}.TW"
    r = requests.get(url)

    soup = bs(r.text, "lxml")

    # 漲跌顏色箭頭對照表
    up_down_mapping = {
        "border-color:transparent transparent #ff333a transparent;border-width:0 6.5px 9px 6.5px": "漲",
        "border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px": "跌"
    }

    # ---------- 股名 ----------
    title_element = soup.find("h1", class_="C($c-link-text) Fw(b) Fz(24px) Mend(8px) Whs(nw)")
    title = title_element.text

    # ---------- 股價 ----------
    price_container = soup.find("div", class_="D(f) Ai(fe) Mb(4px)")
    # print(price_container)
    price = price_container.select_one(".Fz\\(32px\\)").text

    # ---------- 漲跌方向 ----------
    up_or_down_style = soup.find("span", class_="Mend(4px) Bds(s)")['style']
    # print(up_or_down_style)

    up_or_down = up_down_mapping.get(up_or_down_style, "未知")

    # ---------- 漲跌幅 ----------
    percent = price_container.select_one(".Jc\\(fe\\).Fz\\(20px\\)").text

    print(f"{title}: {price} {up_or_down} ({percent})")




In [75]:

# ---------- 多檔股票測試 ----------
stock_codes = ["2330", "2317", "0050", "2408"]

for stock_code in stock_codes:
    get_stock_info(stock_code)

台積電: 1,430 跌 ((2.05%))
鴻海: 241.0 跌 ((4.37%))
元大台灣50: 61.70 跌 ((1.91%))
南亞科: 158.5 跌 ((3.06%))


In [64]:
import requests
from bs4 import BeautifulSoup as bs
import re


def get_stock_info(code):
    url = f"https://tw.stock.yahoo.com/quote/{code}.TW"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    r = requests.get(url, headers=headers)
    r.raise_for_status()

    soup = bs(r.text, "lxml")

    # ---------- 正確股名：從 <title> 抓 ----------
    # e.g. "<title>台積電(2330.TW) 走勢圖 - Yahoo奇摩股市</title>"
    title_tag = soup.title
    if title_tag:
        full_title = title_tag.get_text(strip=True)
        m_name = re.match(r"^(.*?)\(", full_title)   # 抓到 "(" 前面那段
        stock_name = m_name.group(1) if m_name else full_title
    else:
        stock_name = code

    # ---------- 成交價 / 昨收 ----------
    text = soup.get_text(" ", strip=True)

    m_price = re.search(r"成交\s+([0-9,]+(?:\.\d+)?)", text)
    m_prev  = re.search(r"昨收\s+([0-9,]+(?:\.\d+)?)", text)

    if not m_price or not m_prev:
        print(f"{stock_name}：抓不到成交或昨收")
        return

    price_str = m_price.group(1)
    prev_str  = m_prev.group(1)

    price = float(price_str.replace(",", ""))
    prev  = float(prev_str.replace(",", ""))

    change = price - prev
    pct = (change / prev * 100) if prev != 0 else 0.0

    # ---------- 箭頭 / 漲跌方向 ----------
    if change > 0:
        arrow = "🔺"
        word  = "漲"
        sign  = "+"
    elif change < 0:
        arrow = "🔻"
        word  = "跌"
        sign  = "-"
    else:
        arrow = "⏺"
        word  = "平"
        sign  = ""

    change_str = f"{sign}{abs(change):.2f}"
    pct_str    = f"{sign}{abs(pct):.2f}%"

    print(f"{stock_name}: {price_str} {arrow} {word} {change_str} ({pct_str})")




In [65]:
# ---------- 多檔股票測試 ----------
stock_codes = ["2330", "2317", "0050", "1303"]

for c in stock_codes:
    get_stock_info(c)


台積電: 1,430 🔻 跌 -30.00 (-2.05%)
鴻海: 241.0 🔻 跌 -11.00 (-4.37%)
元大台灣50: 61.70 🔻 跌 -1.20 (-1.91%)
南亞: 55.6 🔺 漲 +1.80 (+3.35%)


In [37]:
import requests
from bs4 import BeautifulSoup as bs
import re

url = "https://tw.stock.yahoo.com/quote/2330.TW"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

r = requests.get(url, headers=headers)
soup = bs(r.text, "lxml")

# ---------- 股名：從 <title> 抓 ----------
page_title = soup.title.get_text(strip=True)   # ex: "台積電(2330.TW) 走勢圖 - Yahoo奇摩股市"
stock_name = page_title.split("(")[0]          # -> "台積電"

# ---------- 整頁文字 ----------
full_text = soup.get_text(" ", strip=True)
# print(full_text[:500])  # 要 debug 再打開看前 500 字

# ---------- 股價：從「成交 1,430」抓 ----------
m_price = re.search(r"成交\s+([\d,]+(?:\.\d+)?)", full_text)
stock_price = m_price.group(1) if m_price else None

# ---------- 漲跌金額（絕對值）：從「漲跌 30.00」抓 ----------
m_change_raw = re.search(r"漲跌\s+([\d,]+(?:\.\d+)?)", full_text)
change_abs = float(m_change_raw.group(1).replace(",", "")) if m_change_raw else None

# ---------- 漲跌幅（含正負）：從「漲跌幅 -0.35%」抓 ----------
m_pct = re.search(r"漲跌幅\s+(-?\d+(?:\.\d+)?)%", full_text)
change_pct = float(m_pct.group(1)) if m_pct else None

# ---------- 根據「漲跌幅」決定方向 & 符號 ----------
arrow = ""
change_value_display = "N/A"
change_pct_display = "N/A"

if change_pct is not None:
    if change_pct < 0:
        arrow = "🔺"
        sign = "+"
    elif change_pct > 0:
        arrow = "🔻"
        sign = "-"
    else:
        arrow = "⏺"
        sign = ""

    if change_abs is not None:
        change_value_display = f"{sign}{change_abs:.2f}"
    change_pct_display = f"{change_pct:.2f}%"

print(f"股名：{stock_name}")
print(f"股價：{stock_price}")
print(f"漲跌：{arrow} {change_value_display}")
print(f"漲跌幅：{change_pct_display}")


股名：台積電
股價：1,430
漲跌：🔻 -30.00
漲跌幅：2.05%


In [33]:
import requests
from bs4 import BeautifulSoup as bs


url = "https://tw.stock.yahoo.com/quote/2317.TW"
r = requests.get(url)

soup = bs(r.text, 'lxml')

price = soup.select_one(".D\\(f\\).Ai\\(fe\\).Mb\\(4px\\)")

price = soup.find('div', class_="D(f) Ai(fe) Mb(4px)")
print(price)

price.find('span', class_="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)").text


<div class="D(f) Ai(fe) Mb(4px)"><span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)">241.0</span><span class="Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-down)"><span class="Mend(4px) Bds(s)" style="border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px"></span>11.00</span><span class="Jc(fe) Fz(20px) Lh(1.2) Fw(b) D(f) Ai(c) C($c-trend-down)">(4.37%)</span></div>


'241.0'

In [76]:
import requests
from bs4 import BeautifulSoup as bs
import re

def get_volume_info(code):
    url = f"https://tw.stock.yahoo.com/quote/{code}.TW"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    r = requests.get(url, headers=headers)
    soup = bs(r.text, "lxml")

    text = soup.get_text(" ", strip=True)

    # 今日成交量
    m_vol_today = re.search(r"成交量\s+([0-9,]+)", text)
    # 昨日成交量
    m_vol_prev = re.search(r"昨量\s+([0-9,]+)", text)

    if not m_vol_today or not m_vol_prev:
        return None, None

    vol_today = int(m_vol_today.group(1).replace(",", ""))
    vol_prev  = int(m_vol_prev.group(1).replace(",", ""))

    return vol_today, vol_prev


def detect_volume_spike(stock_codes, factor=5, min_volume=3000):
    results = []

    for code in stock_codes:
        vol_today, vol_prev = get_volume_info(code)
        if vol_today is None:
            continue

        # 是否爆量？
        spike = (vol_today >= vol_prev * factor) and (vol_today >= min_volume)

        results.append((code, vol_today, vol_prev, spike))

    return results


# ---------- 你的股票池（可改成上百檔） ----------
watch_list = [
    "2330", "2317", "2454", "1303", "2882", "2603", "2609",
    "0050", "00929", "2379", "1101"
]


# ---------- 執行爆量偵測 ----------
spike_results = detect_volume_spike(watch_list)

for code, today, prev, is_spike in spike_results:
    tag = "🔥 爆量" if is_spike else "—"
    print(f"{code}: 今日 {today:,} 昨日 {prev:,} → {tag}")


2330: 今日 23 昨日 21,198 → —
2317: 今日 17 昨日 89,283 → —
2454: 今日 18 昨日 2,862 → —
2882: 今日 12 昨日 30,894 → —
2603: 今日 4 昨日 12,756 → —
2609: 今日 6 昨日 12,242 → —
2379: 今日 16 昨日 1,793 → —
